## Metadata VDB Search
We will use Milvus and Towhee to help searches. Towhee is used to extract the semantics of the text and return the text embedding. The Milvus vector database can store and search vectors, and return related dataset's metadata. So we first need to install [Milvus](https://github.com/milvus-io/milvus) and [Towhee](https://github.com/towhee-io/towhee).

Before getting started, please make sure that you have started a [Milvus service](https://milvus.io/docs/install_standalone-docker.md). This notebook uses [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

### Package installations

In [1]:
#! pip install --upgrade pip
#! pip3 install -q towhee pymilvus==2.2.11
#! pip3 uninstall pymilvus -y

! pip3 install -q towhee pymilvus==2.1.1
! pip3 show pymilvus | grep -Ei 'Name:|Version:'
! pip3 show towhee | grep -Ei 'Name:|Version:'

Name: pymilvus
Version: 2.1.1
Name: towhee
Version: 1.1.3


## 1.1 Adding embeddings for columns

The dataset is from the [Kartverket dataset metadata](https://cdn.discordapp.com/attachments/1204433663035449384/1206537816654356480/metadata_no_format.csv?ex=65dc5ee7&is=65c9e9e7&hm=3b9a88db41103ef5393294c5eaeebb60ee2229f43724cc014d4cffc92de1f384&), which contains metadata about each dataset.

The strings in the columns need to be converted to vector representations (embedding) using Towhee [text_embedding.dpr operator](https://towhee.io/text-embedding/dpr). Columns containing these new embedings should contain the original column name with `_vector` at the end.

## 1.2 Load dataset and vectorise chosen column

In [2]:
import pandas as pd
from towhee import pipe, ops, DataCollection
from tqdm import tqdm


# Function to compute embeddings for a single text
def compute_embeddings(text):
    MAX_TOKENS = 512 # Temporary limit on number characters to convert
    truncated_text = text[:MAX_TOKENS]
    return DataCollection(embeddings_pipe(truncated_text)).to_list()[0]['vec']


# Loads dataset into dataframe and recasts columns into correct datatypes
df_kartverket = pd.read_excel('Metadata_excel.xlsx')
recast_to_string = ['datasetcreationdate', 'metadatacreationdate']
df_kartverket[recast_to_string] = df_kartverket[recast_to_string].astype('object')

# Fill NaN values with an empty string
df_kartverket.fillna('', inplace=True)

# Pipe converting text to embeddings (vectors)
embeddings_pipe = (
    pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .output('vec')
)

# Process each column and create new columns for embeddings
column_to_vectorise = 'title'
df_kartverket[column_to_vectorise + '_vector'] = df_kartverket[column_to_vectorise].apply(compute_embeddings)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## 2. Creation of Milvus collection for metadata

In [13]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

server_host = 'ebjerk.no'
server_port = '19530'

connections.connect(host=server_host, port=server_port)

def kartverket_create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
            #FieldSchema(name='schema', dtype=DataType.STRING, max_length=100),  
            FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False),
            #FieldSchema(name='uuid', dtype=DataType.VARCHAR, max_length=100), 
            #FieldSchema(name='hierarchyLevel', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='hierarchyLevel_vector', dtype=DataType.FLOAT_VECTOR, dim=dim), 
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=100),   
            FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=dim),

            #FieldSchema(name='datasetcreationdate', dtype=DataType.VARCHAR, max_length=500),    
            FieldSchema(name='abstract', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='abstract_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='keyword', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='keyword_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='geoBox', dtype=DataType.VARCHAR, max_length=100),    
            #FieldSchema(name='geoBox_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='Constraints', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='Constraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            FieldSchema(name='SecurityConstraints', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='SecurityConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='LegalConstraints', dtype=DataType.VARCHAR, max_length=2000),   
            #FieldSchema(name='LegalConstraints_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            #FieldSchema(name='temporalExtent', dtype=DataType.VARCHAR, max_length=100),    
            ##FieldSchema(name='temporalExtent_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            #FieldSchema(name='image', dtype=DataType.VARCHAR, max_length=1000),    
            FieldSchema(name='responsibleParty', dtype=DataType.VARCHAR, max_length=500),   
            #FieldSchema(name='responsibleParty_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   

            #FieldSchema(name='link', dtype=DataType.VARCHAR, max_length=500),    
            #FieldSchema(name='metadatacreationdate', dtype=DataType.VARCHAR, max_length=500),    
            ##FieldSchema(name='metadatacreationdate_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),    
            FieldSchema(name='productInformation', dtype=DataType.VARCHAR, max_length=1000),   
            #FieldSchema(name='productInformation_vector', dtype=DataType.FLOAT_VECTOR, dim=dim),   
            FieldSchema(name='parentId', dtype=DataType.VARCHAR, max_length=100),   
    ]
    schema = CollectionSchema(fields=fields, description='search text')
    collection_columns = [field_schema.name for field_schema in schema.fields]
    collection = Collection(name=collection_name, schema=schema)
    
    index_params = {
        'metric_type': "L2",
        'index_type': "IVF_FLAT",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name='title_vector', index_params=index_params)
    return collection, collection_columns

kartverket_collection, collection_columns = kartverket_create_milvus_collection('kartverket_metadata', 768)

### 2.1 Creation of dataframe subset to exclude columns with complex data types

In [9]:
#selected_columns = ['id', 'title', 'title_vector', 'abstract', 'keyword', 'Constraints', 'SecurityConstraints', 'LegalConstraints', 'responsibleParty', 'productInformation', 'parentId']
df_kartverket_slice = df_kartverket[collection_columns]
df_kartverket_slice

AttributeError: 'Collection' object has no attribute 'fields'

### 2.2 Insert the subset dataframe data into Milvus collection

In [5]:
from towhee import ops, pipe, DataCollection

insert_pipe_kartverket = (pipe.input('df_kartverket_slice')
                   .flat_map('df_kartverket_slice', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name='kartverket_metadata'))
                   .output('res')
)

%time _ = insert_pipe_kartverket(df_kartverket_slice)


kartverket_collection.load()
kartverket_collection.num_entities

CPU times: user 709 ms, sys: 87.7 ms, total: 797 ms
Wall time: 8.01 s


189

## 3. Prepare pipe that accepts queries

In [6]:
import numpy as np
# Variables specifying what column and collection to perform ANN comparrison against
vector_columns = ['title_vector']
collection_name = 'kartverket_metadata'

print(df_kartverket.columns)
# What columns to return for view
response_output = [
       'id', 'id', 'title', # required to have two 'id'
       'abstract', 'keyword', 'Constraints',
       'SecurityConstraints', 'LegalConstraints',
       'responsibleParty', 'productInformation', 'parentId'
]


demo_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'id', 'title', # required to have two 'id'
                            'abstract', 'keyword', 'Constraints',
                            'SecurityConstraints', 'LegalConstraints',
                            'responsibleParty', 'productInformation', 'parentId'), 
                                       ops.ann_search.milvus_client(host=server_host, 
                                                                    port=server_port,
                                                                    collection_name=collection_name,
                                                                    vector_field=vector_columns,
                                                                    output_fields=response_output, 
                                                                    limit=5))  
                    .output(*['query', 'score'], *response_output)
               )

Index(['schema', 'uuid', 'id', 'hierarchyLevel', 'title',
       'datasetcreationdate', 'abstract', 'keyword', 'geoBox', 'Constraints',
       'SecurityConstraints', 'LegalConstraints', 'temporalExtent', 'image',
       'responsibleParty', 'link', 'metadatacreationdate',
       'productInformation', 'parentId', 'title_vector'],
      dtype='object')


## 3.1 Query against metadata collection

In [7]:
kartverket_question1 = 'Hvor er bomberom?'
print(f'\n"{kartverket_question1}" search:')
res_kartverket1 = demo_pipe(kartverket_question1)
DataCollection(res_kartverket1).show()


"Hvor er bomberom?" search:


## 4. Format response into GPT API request
After the query is vectorised, and then compared against the database's vector column to find the most simmilar result(s). The most similar result's metadata can be formatted into a instruction with example for GPT. This way, we can instruct GPT how to answer the query by providing a demonstration. Before prompting with the query's current context, and the instruction to answer.

![alt text](image.png)

In [8]:
metadata_demonstration = '''query	score	id	id	title	abstract	keyword	Constraints	SecurityConstraints	LegalConstraints	responsibleParty	productInformation	parentId
Hvor er bomberom?	118.3987045288086	22788	22788	Magasin	Database over regulerte innsjøer. Egenskapsdata er vassdragsnr., magasinnr., navn, laveste og høyeste regulerte vannstand (m.o.h...	Annet###Åpne data###Norge digitalt###modellbaserteVegprosjekter###fellesDatakatalog###Energi###Norge fastland###magasin###vannkr...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Seming Haakon SkauNorges vassdrags- og energidirektoratgisstotte@nve.no###NVE - Seksjon for geomatikk og dataanalyse/IGDNorges v...	Produktspesifikasjon###Produktark###Produktside###Tegnforklaring###dekningsoversikt###hjelp###dekningsoversikt rutenett###deknin...
Hvor er bomberom?	123.4372329711914	21492	21492	Bunnsedimenter (kornstørrelse), detaljert	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del (øverste 0-10 cm av sjøbunnen). I egenskapstabellen...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###ØkologiskGrunnkart###MarineGrunnkart###fellesDatakatalog###Geo...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:20.000 - 1:150.000. ###Detaljnivået på d...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	124.96815490722656	68030	68030	Bunnsedimenter (kornstørrelse), oversikt	Datasettet viser kornstørrelsessammensetning i sjøbunnssedimentenes øvre del. Kornstørrelsesdata er basert på analyser av sjøbun...	Sea regions###Inspire###Norge digitalt###geodataloven###Mareano###fellesDatakatalog###Geologi###Norge###Nordsjøen###Norskehavet#...	Bruksbegrensninger Detaljnivået på datasettet tilsier bruk innenfor kartmålestokken: 1:2000.000 - 1:10.000.000 ###Detaljnivået ...	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator4773904000Leiv Eirikssons vei 39TrondheimAave.Lepland@ngu.nohttp://ww...	https://register.geonorge.no/produktspesifikasjoner/bunnsedimenter-kornstørrelseProduktspesifikasjon###https://register.geonorge...
Hvor er bomberom?	125.0992431640625	75832	75832	Vegetasjon - Naturtyper	Kommunvise kartlag i datasettet Vegetasjon som viser naturtyper som er viktig for biologisk mangfold og som helt eller delvis ka...	Land cover###Åpne data###Norge digitalt###fellesDatakatalog###Landbruk###Natur###Norge fastland###Vegetasjon###Arealdekke###Vege...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Ingvild NystuenNorsk institutt for bioøkonomiSeksjonsleder64 94 80 0064 94 80 01Raveien 9ÅsAkersjusPostboks 115, 1430 ÅsNorgegis...	http://www.skogoglandskap.no/seksjoner/nedlastingsinformasjonNedlastingsinformasjon###http://www.skogoglandskap.no/kart/vegetasj...
Hvor er bomberom?	125.26884460449219	69607	69607	Relativ bunnhardhet, åpne data	Relativ bunnhardhet er rasterdata som viser reflektivitetstyrke fra sjøbunnen. Reflektivitetsstyrke sier noe om sjøbunnens akust...	Geology###Åpne data###Norge digitalt###MarineGrunnkart###modellbaserteVegprosjekter###fellesDatakatalog###Geologi###Norge###Bare...	Bruksbegrensninger Ingen begrensninger på bruk er oppgitt. ###Ingen begrensninger på bruk er oppgitt.No conditions apply	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Aave LeplandNorges geologiske undersøkelseDataadministrator, Maringeologi, NGU4773904000Leiv Eirikssons vei 39TrondheimAave.Lepl...	Produktspesifikasjon###https://register.geonorge.no/register/versjoner/produktark/norges-geologiske-undersokelse/relativ-bunnhar...'''
metadata_link = 'https://ipt.nina.no/'

query = f'Finn dataset med magasin'
instruction_gpt = f'Skriv en respons som finner det mest korresponderende datasettet fra metadata for spørringen:'
demonstration = f'Spørring: Hvor er bomberom?\nMetadata: {metadata_demonstration}\nRespons: Ut i fra spørringen ser det ut som at du leter etter datasettet Tilfluktsrom (nedlastningslink: {metadata_link}). Dette datsettet inneholder informasjon om lokasjonene til alle offentlige tilfluktsrom (bomberom) i Norge'
current_query_instruction = f'Spørring:{query}\nMetadata: {metadata_demonstration}\nRespons:'
API_text = f'{instruction_gpt}\n{demonstration}\n\n{current_query_instruction}'
print(API_text)

Skriv en respons som finner det mest korresponderende datasettet fra metadata for spørringen:
Spørring: Hvor er bomberom?
Metadata: query	score	id	id	title	abstract	keyword	Constraints	SecurityConstraints	LegalConstraints	responsibleParty	productInformation	parentId
Hvor er bomberom?	118.3987045288086	22788	22788	Magasin	Database over regulerte innsjøer. Egenskapsdata er vassdragsnr., magasinnr., navn, laveste og høyeste regulerte vannstand (m.o.h...	Annet###Åpne data###Norge digitalt###modellbaserteVegprosjekter###fellesDatakatalog###Energi###Norge fastland###magasin###vannkr...	Bruksbegrensninger Ingen ###Ingen	Sikkerhetsnivå Ugradert: Available for general disclosure#########	Tilgangsrestriksjoner Andre restriksjoner: Limitation not listed######Andre restriksjonerÅpne data###Åpne data###Brukerrestriksj...	Seming Haakon SkauNorges vassdrags- og energidirektoratgisstotte@nve.no###NVE - Seksjon for geomatikk og dataanalyse/IGDNorges v...	Produktspesifikasjon###Produktark###Produktside#